# Calculate the distorsion polynomial coefficients

Determines a common model for all test inputs
Outputs the model coefficients

In [1]:
%matplotlib notebook

import sys
import logging
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

sys.path.append("../DesignTool")
from smdtLibs.configFile import ConfigFile
from targets import TargetList
from smdtLibs.inOutChecker import InOutChecker
from maskDesignFile import (
    MaskDesignInputFitsFile,
    MaskDesignOutputFitsFile,
    outputAsList,
)
from maskLayouts import MaskLayouts, GuiderFOVs, shrinkMask
from smdtLibs import utils, drawUtils
from astropy.modeling import models, fitting

from DistortionUtil import calcDistortionCoef

#from Test_InputM import Test_Inputs
from Test_Input1 import Test_Inputs

In [2]:
try:
    %load_ext nb_black
except:
    print("Ignore extension nb_black")
    pass

<IPython.core.display.Javascript object>

In [3]:
logging.disable()

<IPython.core.display.Javascript object>

## Read the config file and the parameter file

In [4]:
config_fname = "../DesignTool/smdt.cfg"
param_fname = "../DesignTool/params.cfg"

config = ConfigFile(config_fname)
config.properties["params"] = ConfigFile(param_fname)

<IPython.core.display.Javascript object>

## The mask layout

In [5]:
layout = MaskLayouts["deimos"]
guiderLayout = GuiderFOVs["deimos"]

<IPython.core.display.Javascript object>

In [6]:
def coeffNames(prefix, n):
    """
    Generates coefficient names
    """
    out = [prefix + "00"]
    out.extend([f"{prefix}x{i}y0" for i in range(1, n + 1)])
    out.extend([f"{prefix}x0y{i}" for i in range(1, n + 1)])
    for y in range(1, n):
        for x in range(1, n - y + 1):
            out.append(f"{prefix}x{x}y{y}")
    return out

<IPython.core.display.Javascript object>

In [19]:
# Determines distortion coefficients for enabled input files.
coeffs = []

for k, item in Test_Inputs.items():
    (input_fname, input_RA, input_DEC, fieldPA, hourAngle, enabled) = item
    if not enabled:
        continue
    mdf, (xfit, yfit) = calcDistortionCoef(item, config, layout, -1, 178.5)

    a = [
        k,
        mdf.maskblu.RefWave[0],
        hourAngle,
        fieldPA,
    ]  # Reference wavelenght
    # XFit parameters (15)
    a.extend(list(xfit.parameters))
    # YFit parameters (15)
    a.extend(yfit.parameters)
    coeffs.append(a)
    # print(", ".join([f"{x}" for x in a]))

<IPython.core.display.Javascript object>

In [20]:
#
# Outputs results
#
colNames = ["Name", "RefWave", "HA", "PA"]
colNames.extend(coeffNames("X_", 4))
colNames.extend(coeffNames("Y_", 4))

coeffsDf = pd.DataFrame(coeffs, columns=colNames)
coeffsDf[["Name", "RefWave", "HA", "PA", "X_00", "Y_00"]]

,Name,RefWave,HA,PA,X_00,Y_00
0,dec0Pa0,320.0,0.015,0.0,350.164617,460.175915
1,dec0Pa105,320.0,0.015,105.0,291.098630,98.066426
2,dec0Pa120,320.0,0.015,120.0,-0.959443,-0.025448
3,dec0Pa135,320.0,0.015,135.0,-1.032966,-0.066617
4,dec0Pa150,320.0,0.015,150.0,-1.007216,-0.028267
5,dec0Pa15,320.0,0.015,15.0,201.975037,298.279707
6,dec0Pa165,320.0,0.015,165.0,-1.008034,0.039748
7,dec0Pa180,320.0,0.015,180.0,-1.002314,-0.041171
8,dec0Pa30,320.0,0.015,30.0,120.685341,165.262481
9,dec0Pa45,320.0,0.015,45.0,163.038342,149.071784


<IPython.core.display.Javascript object>

In [9]:
#
# Averages the coefficients
#
startidx = 2
xs = [f"{coeffsDf[cn].mean()}" for cn in colNames[startidx : startidx + 15]]
ys = [f"{coeffsDf[cn].mean()}" for cn in colNames[startidx + 15 :]]

print('distortionXCoeffs = "', ", ".join(xs), '"')
print('distortionYCoeffs = "', ", ".join(ys), '"')
#
# The output should be copied to smdt.cfg
#

distortionXCoeffs = " 0.015000000000000005, 90.0, 113.69413285020211, 0.7789555154902353, 1.1497251565109003e-05, 3.936977228180445e-09, -1.827561099358528e-11, -1.3113472166384843, 0.005368285028252672, -9.349550586087523e-06, 5.8438248732431355e-09, 0.0021872830654965096, -6.797470952423714e-06, 6.791012723637494e-09, -4.4703874004820445e-08 "
distortionYCoeffs = " 7.431722094700687e-11, 1.688226614852921e-11, 121.93020079735531, -0.12471085585563742, 3.9047174293852694e-05, 4.351661312369696e-09, -1.8207774694731938e-11, -0.3747035085257905, 0.005613722160842638, -9.64640900420385e-06, 5.901667936596928e-09, 0.0011840339706207439, -3.615280448936978e-06, 3.5740167068527536e-09, -1.2072464370064275e-07, 1.9346286756488762e-10, -2.841291949121306e-11 "


<IPython.core.display.Javascript object>

In [10]:
mdf = MaskDesignInputFitsFile("../DeimosExamples/Examples1/dec0Pa105.fits")

<IPython.core.display.Javascript object>

In [11]:
mdf.maskdesign

,DesId,DesName,DesAuth,DesCreat,DesDate,DesNslit,DesNobj,ProjName,INSTRUME,MaskType,RA_PNT,DEC_PNT,RADEPNT,EQUINPNT,PA_PNT,DATE_PNT,LST_PNT
0,1,...,sy <syeh@keck.hawaii.edu> ...,Dsimulator: Ver 0.0b ...,2021-03-23T00:53:59,451,455,test ...,DEIMOS ...,??? ...,247.358387,0.850802,,2000.0,105.0,2021-03-25,0.015


<IPython.core.display.Javascript object>

In [15]:
mdf.maskblu

,BluId,DesId,BluName,guiname,BluObsvr,BluCreat,BluDate,LST_Use,Date_Use,TELESCOP,RefrAlg,AtmTempC,AtmPres,AtmHumid,AtmTTLap,RefWave,DistMeth
0,1,1,...,test,sy <syeh@keck.hawaii.edu> ...,DSIMULATOR -- 02Jul25 ...,2021-03-23T00:53:59,16.493,2021-03-25,Keck II ...,SLALIB ...,0.0,615.0,0.4,0.0065,320.0,INDEF ...


<IPython.core.display.Javascript object>

In [13]:
16.493 / 15

1.0995333333333333

<IPython.core.display.Javascript object>